# Demo notebook

## 1. THE BASICS

The first thing you will need to do in order to use plots is **to import them from the sisl.viz module**:

In [ ]:
from sisl.viz import Plot

*Let's start plotting!*

**Why can't I just tell the computer to plot the file?!** This is the question milions of students around the world ask themselves when they get introduced to computational physics. Then time passes, and you learn to deal with it. You might even build your own scripts in an attempt to mitigate the pain. But hey:

In [ ]:
Plot("../files/SrTiO3.bands")

<span class="user">*Wow, so impressive, you have a script to plot bands...*</span>

In [ ]:
Plot("../files/SrTiO3.RHO", axes=[0,1], sc=[2,1,1], zsmooth='best')

See what we did there? We passed some extra arguments (`axes` and `sc`) to get the exact plot that we want.

<span class="user">But how do you know which words to use?</span>

Be patient, we will get there.

<span class="user">Ok, but still, you only have different scripts that plot different things. And you use this fancy `Plot()` stuff to catch our attention.<span class="user">

Not really. In fact, what `Plot()` does is to initialize a plot instance. If you store it into a variable you can check for yourself.

In [ ]:
plot = Plot("../files/SrTiO3.RHO", axes=[0,1], sc=[2,1,1], zsmooth="best")
plot.__class__

In [ ]:
plot.show("png")

Note also that it's not exactly a `Plot`, but a `GridPlot`. This means that it will have **settings and methods that are specific** to this kind of plot.

<span class="user">Settings?</span>

Yes, in this visualization framework plots have settings. When settings change, the plot changes. But wait a bit for that. 

*You can `print(plot)` to see how they look like if you really can't wait. Here's a cell for you, we won't know:*

Are you done? Good. Now if you want to make a better use of that cell do `help(plot)` instead. You will see a list of the mehods that are available to you in this plot. Actually we are more interested in the specific methods that this plot has, so when you reach the line:

`Methods inherited from sisl.viz.plot.Plot:`

you can stop reading.

*Let's try one of those!*

In [ ]:
# We already have a plot stored
plot.scan(steps=15)

<span class="user">Ok, now it starts to make sense... but looking at the help message is a bit tedious, don't you think?</span>

Yes, I agree. The plan is to have an individual notebook to showcase the abilities of each plot, because, you know, we came here to visualize. But we're not quite there yet, sorry.

<span class="user">Btw... why did an animation come out of nowhere?</span>

Magic doesn't exist, so of course someone has developed it.

<span class="user">I'm sure it's complicated as hell to use. This is why you've been hiding it from us...</span>

Well, you can judge for yourself.

In [ ]:
from sisl.viz import Animation, BandsPlot

# There are two main ways to build an animation for yourself

# You can either provide already initialized plots to Animation
plots = [ Plot("../files/SrTiO3.bands", bands_color=color) for color in ("green", "orange", "red")]

from_animation = Animation(plots=plots)

# Or use the animated class method
from_animated = BandsPlot.animated(
    "bands_color", ("green", "orange", "red"), # Animate the bands_color setting with this values
    fixed = {"bands_file": "../files/SrTiO3.bands"}, # Keep this settings fixed for all frames
)

# Show them both
from_animation.show()
from_animated.show()

So, you basically get the same animation in both cases. But is it really the same?

In [ ]:
# Let's save them
from_animation.save("From_animation.plot")
from_animated.save("From_animated.plot")

If you go to the directory where this tutorials are and you run the command `ll -h` you will see the following (moreless):
- *From_animated.plot* weights **679 KB**
- *From_animation.plot* weights **855 KB**

I'll give you a moment to solve the mistery...

That's right! Animation has detected that some data can be shared between plots and it is doing so.

<span class="user">But how?!</span>

Again, settings. It's always about settings.

But you must know that this is a crucial feature for plots with larger data, like the charge density scan we saw previously. In that case, your computer might not even be able to sustain such HUGE quantities of data. Therefore, there is a `template_plot` keyword that you can provide to both `animated` and `Animation` to explicitly set the plot that will share data.

By the way, here's a cell for you to check that I'm not lying, try checking the `shared` attribute of the plot that came from the `animated` method.

<span class="user">Ok, I see. Can you tell us how to load the plots that you saved there?</span>

Sure:

In [ ]:
from sisl.viz import load
#load("I know you have an intuition of what to put here")

<span class="user">Thanks, but what's the point of saving them anyway?</span>

Well, you can:
- Open them later and they will already contain all the data.
- Share it with other people easily.

However, if you just want to share the plot and not the data, you have beter and more lightweight options:
- Save it as a picture (you probably already saw the picture icon that appears when you hover the plot at the top-right corner)
- Save it as html using the `.html()` method: Great for keeping functionality and vital for animations.
- Take the `figure` attribute and save it however you want.

This last point brings me to a breakpoint in our relation. I have to make a confession: *I haven't developed the figures themselves, these are [plotly](https://plotly.com/) figures.*

<span class="user">Aha! I knew there was something you were hiding.</span>

Yes, indeed. I was hiding that, since `Plot` is an extension of plotly `Figure`, all plotly methods are available to you to postprocess the plot that you obtained as you wish:

In [ ]:
# Let's build a plot
plot = Plot("../files/SrTiO3.RHO", axes=[0,1], sc=[2,1,1], zsmooth="best", title="The importance of the green line")

# And now let's draw a line a line indicating something super important
# using the plotly add_scatter method on our figure
plot.add_scatter(x=[0,4], y=[1,3],line_width=5, line_color="lightgreen", showlegend=False)

*Now you got a taste of the basic things that you should expect from sisl's visualization module. We hope that you liked what you saw :)*

If you are willing to know more, please keep on reading the notebook. **We appreciate your interest and we are definitely willing to show more**. And we honestly think you won't regret it. 

## 2. LET'S TALK ABOUT SETTINGS

<span class="user">Finally!</span>

*Life is unpredictable.*

And that's why needing to define all the parameters of your plot on initialization seems like too much to ask. That's why, we've separated the work of our plots into three main processes:

- Reading the data.
- Processing the data.
- Displaying the figure.

At this point, you probably already see that there is clearly at least one step that you don't want to be doing all the time: *Reading your 300MB file.*

<span class="user">I get it, I can run different steps to update my plot as I need. But how would I know, if it was not me that developed the plot? Do I have to look into the code of each plot?!</span>

No, that's the point of settings. Plots have settings (*as you have previously discovered, I know what you did in that cell...*). And plots know which settings are run in each step of the process. So, when you update a setting, don't worry, they will know what to do.

<span class="user">And how do I "update a setting"?</span>

In [ ]:
# Get the bands plot
plot = Plot("../files/SrTiO3.bands")

# If you don't know what are the settings that you can change, print the plot
# print(plot)

# But I already know, so I will update them using the update_settings method
plot.update_settings(Erange=[-5,5], gap=True)

<span class="user">And what if I messed up the plot with the new settings?</span>

In [ ]:
plot.undo_settings()

The `undo_settings()` method even has a `nsteps` keyword that will allow you to go way back. No one will ever know about your "little experiments".

## 3. MONITORING CHANGES 

Plots can keep track of the files that they read.

Using this, you can check if there are updates available for a given plot very easily. 

In [ ]:
plot = Plot("../files/SrTiO3.bands")
plot.updates_available()

<span class="user">Of course there are no available updates, you just read the file...</span>

Yeah, thanks.

<span class="user">Do I have to be checking for updates until there is one? That's pointless.<span>
    
Well, you can, sure, if you want. But of course there is a method already there for you.

Let's build a different plot that makes sense tracking.

In [ ]:
from sisl.viz import ForcesPlot
forces = ForcesPlot(out_file="../files/water/water.out")

<span class="user">Oops! Seems that these plots are not that great after all.<span>

We are trying to make a plot out of a file that doesn't even exist. So, the error is fair enough.
    
Although it does not yet exist, Lets make the plot listen for updates:

In [ ]:
forces.listen()

<span class="user">Wow, impressive. Seems like it didn't do much...</span>
    
Well, that's the point of the `listen` method. It runs *asyncronously*. This means that it does not block the execution of other code. It basically means that, while this plot is listening for updates, **you can continue doing things on the notebook**.

But enough talk, let's see it working. Go to the `files/water` folder and execute siesta so that it sends the output to a file called `water.out`:

`
siesta < water.fdf > water.out
`

Let's see what happens...

Did you see how the plot was changing? The plot does not know when to stop listening. Who knows, maybe an update will come sometime. Let's take a moment to stop it.

In [ ]:
forces.stop_listening()

You probably saw that the updating process didn't look great. That is because we need to clear the output and plot the figure again each time.

<span class="user"> So classy... </span>

Well, the plot is only doing that because it does not find the appropiate jupyter notebook widgets. Check which widgets are you missing:

In [ ]:
forces._widgets

If you have plotly available but it indicates an error, it is probably because, although you are in your environment's kernel, you are running `jupyter notebook` from a different environment.

Just run the notebook from your environment, or uncomment this cell and run it (don't remove the !) to install plotly also in the environment that you are running the notebook.

In [ ]:
#!pip3 install plotly

Then close and open the jupyter server again with `jupyter notebook` or however you do it (**restarting the kernel is not enough**).

Regarding `events`, it is a widget that will provide you with the ability to **use keyboard shortcuts on the plots**. It can be very handy, but if you don't want it, that's fine.

If you want to have it:

In [ ]:
#!pip3 install ipyevents
#!jupyter nbextension enable --py --user ipyevents

Or with conda:

In [ ]:
#!conda install -c conda-forge ipyevents

If you installed event, let's take a moment to see if it works.

Close the server and open it again with `jupyter notebook` or however you like to do it (**restarting the kernel is probably not enough**).

In [ ]:
plot = Plot("../files/SrTiO3.RHO")
plot.show()

Try to play with it and see if it works.

Place your mouse on top of the plot and press any key. You should see it logged in the left-down corner.

Then press `shift+?` to see all the available shortcuts for this plot. Try to use them. If you've gotten here, there should be no problem now :)

Fun part is that you don't need to stick to the shortcuts that are provided by default, you can build your own so that it fits perfectly your requirements. More on this in upcoming tutorials. But feel free to add a little pressure so that we do it faster, we will appreciate your enthusiasm for sure.